In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/mount")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install torch > /dev/null
! pip install transformers > /dev/null
! pip install datasets > /dev/null

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# load model
# tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/mount/model")
# model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/Colab Notebooks/mount/model").to('cuda')

model_name = 'model_v5_EP8_2023_1130_1450_10'

tokenizer = AutoTokenizer.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/mount/V5/{model_name}")
model = AutoModelForCausalLM.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/mount/V5/{model_name}").to('cuda')

PAD_IDX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
IGNORED_PAD_IDX = -100
PAD_IDX



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


50278

In [ ]:
from datasets import load_dataset, Features, Value
valid_data = load_dataset(
          "csv", data_files="opendid_test.tsv",
          delimiter='\t',
          features = Features({'fid': Value('string'), 'idx': Value('int64'),'content': Value('string'), 'label': Value('string')}),
          column_names=['fid', 'idx', 'content', 'label']
        )

valid_list= list(valid_data['train'])
valid_list[:2]

[{'fid': '1097', 'idx': 1, 'content': '433475.RDC', 'label': None},
 {'fid': '1097', 'idx': 12, 'content': 'Timmins, ELDEN', 'label': None}]

In [ ]:
from tqdm import tqdm#, tqdm_notebook
import torch
import datasets
bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep ='\n\n####\n\n'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer.padding_side = 'left'
def sample_batch(model, tokenizer, input):
    """Generate text from a trained model."""
    model.eval()
    seeds = [f"{bos} {text['content']} {sep}" for text in input]
    texts = tokenizer(seeds, return_tensors = 'pt', padding=True).to(device)
    outputs = []
    #return
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**texts, max_new_tokens=400, pad_token_id = PAD_IDX,
                                        eos_token_id=tokenizer.convert_tokens_to_ids(eos))
        preds = tokenizer.batch_decode(output_tokens)
        for idx , pred in enumerate(preds):
            pred = pred[pred.index(sep)+len(sep):].replace(pad, "").replace(eos, "").strip()
            if pred == "PHI: NULL":
                continue
            phis = pred.split('\n')
            lidxs = {}
            for p in phis:
                tid = p.find(':')
                if tid > 0:
                    text = p[tid+1:].strip()
                    nv = text.find('=>')
                    normalizedV = None

                    if nv>0:
                      normalizedV = text[nv+2:]
                      text = text[:nv]

                    lidx = 0
                    if text in lidxs:
                        lidx = lidxs[text]
                    lidx = input[idx]['content'].find(text, lidx)
                    eidx = lidx+len(text)
                    lidxs[text] = eidx
                    sidx=int(input[idx]['idx'])
                    if len(text) > 0 and p[:tid] in ['DOCTOR','DATE','IDNUM','MEDICALRECORD','PATIENT','HOSPITAL','TIME','DEPARTMENT','CITY','ZIP','STREET','STATE','AGE','ORGANIZATION','DURATION','PHONE','URL','LOCATION-OTHER','SET','COUNTRY','ROOM']:
                      if normalizedV is None:
                          outputs.append(f'{input[idx]["fid"]}\t{p[:tid]}\t{lidx+sidx}\t{eidx+sidx}\t{text}')
                      else:
                          outputs.append(f'{input[idx]["fid"]}\t{p[:tid]}\t{lidx+sidx}\t{eidx+sidx}\t{text}\t{normalizedV}')
    return outputs
# utf8
# len(valid_list)
f = open("answer.txt", "w" , encoding='utf-8')
# BATCH_SIZE = 400 # A100
BATCH_SIZE = 100 # V100
# BATCH_SIZE = 100  #T4
for i in tqdm(range(0, len(valid_list) , BATCH_SIZE)):
    with torch.no_grad():
        seeds = valid_list[i:i+BATCH_SIZE]
        outputs = sample_batch(model, tokenizer, input=seeds)
        for o in outputs:
            f.write(o)
            f.write('\n')
f.close()

100%|██████████| 790/790 [09:24<00:00,  1.40it/s]
